In [1]:
# make sure ES is up and running
import requests
res = requests.get('http://localhost:9200')
print(res.content)


{
  "name" : "idAwBkj",
  "cluster_name" : "elasticsearch",
  "cluster_uuid" : "GVeWRJgNRZubjiNgz5FQlQ",
  "version" : {
    "number" : "5.4.0",
    "build_hash" : "780f8c4",
    "build_date" : "2017-04-28T17:43:27.229Z",
    "build_snapshot" : false,
    "lucene_version" : "6.5.0"
  },
  "tagline" : "You Know, for Search"
}



In [3]:
#connect to our cluster
from elasticsearch import Elasticsearch
es = Elasticsearch([{'host': 'localhost', 'port': 9200}])
print es

<Elasticsearch([{'host': 'localhost', 'port': 9200}])>


In [ ]:
import json
import csv
with open('export.csv') as f:
    reader = csv.DictReader(f)
    rows = list(reader)
    for data in rows:
        es.index(index='news',doc_type='news', body=data)

In [12]:
es.get(index='test', doc_type='text', )

TypeError: get() takes at least 3 arguments (4 given)

In [ ]:
res = es.search(index="news", body={"query": {"match_all": {}}})
print len(res['hits']['hits'])

In [2]:
result = es.search(index="news", body={"query": {"more_like_this": 
                                { "fields" : ["title", "description"],"like_text" : "Trump","min_term_freq" : 1}}})

In [4]:
import json
jsonR = json.dumps(result)

In [19]:
print result['hits']['hits'][0]['_source']

{u'description': u'At least 13 people were arrested as pro-Trump and anti-Trump protestors clashed Saturday at a park in Berkeley, California, police said.', u'title': u'Pro-Trump, anti-Trump protesters clash in Berkeley', u'url': u'http://www.cnn.com/2017/04/15/us/berkeley-protests-trump/index.html', u'author': u'Ralph Ellis and Tony Marco, CNN', u'publishedAt': u'2017-04-15T23:50:56.000Z', u'source': u'cnn', u'urlToImage': u'http://i2.cdn.cnn.com/cnnnext/dam/assets/170415161642-berkeley-protesters-01-super-tease.jpg', u'class': u'Politics & Government', u'digest': u'MrI5ZMh1wrEwPQ9le8GuuQ==\n'}


In [3]:
total_news=[]
for i in result['hits']['hits']:
    total_news.append(i['_source'])
print total_news

[{u'description': u'At least 13 people were arrested as pro-Trump and anti-Trump protestors clashed Saturday at a park in Berkeley, California, police said.', u'title': u'Pro-Trump, anti-Trump protesters clash in Berkeley', u'url': u'http://www.cnn.com/2017/04/15/us/berkeley-protests-trump/index.html', u'author': u'Ralph Ellis and Tony Marco, CNN', u'publishedAt': u'2017-04-15T23:50:56.000Z', u'source': u'cnn', u'urlToImage': u'http://i2.cdn.cnn.com/cnnnext/dam/assets/170415161642-berkeley-protesters-01-super-tease.jpg', u'class': u'Politics & Government', u'digest': u'MrI5ZMh1wrEwPQ9le8GuuQ==\n'}, {u'description': u'One issue Trump supporters and critics agree on: The country remains deeply divided.', u'title': u'The Trump Years', u'url': u'https://www.usatoday.com/pages/interactives/news/the-trump-years/', u'author': u'', u'publishedAt': u'2017-04-26T14:16:51.000Z', u'source': u'usa-today', u'urlToImage': u'https://www.gannett-cdn.com/experiments/usatoday/responsive/2017/04-trump-pre

In [11]:
k=0
for j in total_news:
    j['level']=k
    k=k+1
print total_news

[{u'description': u'At least 13 people were arrested as pro-Trump and anti-Trump protestors clashed Saturday at a park in Berkeley, California, police said.', u'title': u'Pro-Trump, anti-Trump protesters clash in Berkeley', u'url': u'http://www.cnn.com/2017/04/15/us/berkeley-protests-trump/index.html', u'author': u'Ralph Ellis and Tony Marco, CNN', u'publishedAt': u'2017-04-15T23:50:56.000Z', u'source': u'cnn', u'urlToImage': u'http://i2.cdn.cnn.com/cnnnext/dam/assets/170415161642-berkeley-protesters-01-super-tease.jpg', 'level': 0, u'class': u'Politics & Government', u'digest': u'MrI5ZMh1wrEwPQ9le8GuuQ==\n'}, {u'description': u'One issue Trump supporters and critics agree on: The country remains deeply divided.', u'title': u'The Trump Years', u'url': u'https://www.usatoday.com/pages/interactives/news/the-trump-years/', u'author': u'', u'publishedAt': u'2017-04-26T14:16:51.000Z', u'source': u'usa-today', u'urlToImage': u'https://www.gannett-cdn.com/experiments/usatoday/responsive/2017/

In [12]:
for news in total_news:
        #log_client.logger.debug('Create News class [%s] label, Current Top Preference is [%s]' % (news['class'], topPreference))
        #print 'Create News class [%s] label, Current Top Preference is [%s]' % (news['class'], topPreference)
        #remove text field to save bandwidth
        del news['text']
        if news['publishedAt'].date() == datetime.today().date():
            news['time'] = 'today'
        else:
            news['time'] = news['publishedAt'].date().strftime("%A %d. %B %Y")

KeyError: 'text'

In [4]:
es.indices.delete(index='news')

{u'acknowledged': True}